In [1]:
# Code cell intended for imports and global settings.

# Imports
import torch
import torch.nn as nn
import torch.optim as optim
import syft as sy
from SupportClasses import ModelSupport as ms, EnvironmentSetup as env

# Global values
NUMBER_OF_NODES = 5
NUMBER_OF_EPOCHS = 5

# Model specific values
criterion = nn.CrossEntropyLoss()


In [2]:
# This code cell is to be used for importing data and setting up the model.
train_loader, validation_loader, test_loader = env.download_mnist()
# Create and load the model
model = ms.ConvNet(len(train_loader.dataset.dataset.classes))
models = [model]

# Send the models to the CUDA device if it exists.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
for model in models:
    model.to(device=device)

In [3]:
# The main training loop will go here
best_valid_loss = float('inf')
optimizer = optim.Adam(model.parameters())

for epoch in range(NUMBER_OF_EPOCHS):

    train_loss, train_acc = ms.train(model, train_loader, optimizer, criterion, device=device)
    valid_loss, valid_acc = ms.test(model, validation_loader, criterion, device=device)

    # end_time = time.time()
    #
    # epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best-model.pt')

    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.5f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.5f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01
	Train Loss: 0.00504 | Train Acc: 96.06%
	 Val. Loss: 0.00060 |  Val. Acc: 98.16%


KeyboardInterrupt: 

In [4]:
# The main testing loop
model.load_state_dict(torch.load('best-model.pt'))

test_loss, test_acc = ms.test(model, test_loader, criterion, device=device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.000 | Test Acc: 99.01%
